# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Matemática Para Data Science</font>

## <font color='blue'>Estudo de Caso 3</font>
### <font color='blue'>Operações com Matrizes em Redes Neurais Artificiais</font>

![title](CAP06/imagens/EC3.png)

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
# Você pode usar a versão indicada nos vídeos ou a versão abaixo!
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.18


In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# !pip install -q -U watermark

In [4]:
# Imports
import random
import numpy as np
import typing as List

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy: 1.26.3



Abaixo temos uma classe Python que descreve uma rede neural artificial apenas com operações matemáticas. Não se deixe impressionar pelo código, é bem mais simples do que parece!!

Vamos detalhar as operações matemáticas dessa função.

In [10]:
# Classe
class NeuralNetwork(object):

    # Método construtor
    def __init__(self, sizes: list[int]) -> None:    
        
        """A lista `sizes` contém o número de neurônios nas
         respectivas camadas da rede. Por exemplo, se a lista
         for [2, 3, 1] então será uma rede de três camadas, com o
         primeira camada contendo 2 neurônios, a segunda camada 3 neurônios,
         e a terceira camada 1 neurônio. Bias e pesos para a
         rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. 
         Note que a primeira camada é assumida como uma camada de entrada, e por convenção nós
         não definimos nenhum bias para esses neurônios, pois os bias são usados
         na computação das saídas das camadas posteriores."""

        # Número de camadas
        self.num_layers = len(sizes)
        
        # Define o atributos sizes
        self.sizes = sizes
        
        # Cria um vetor de valores randômicos para inicializar o bias. 
        # A função randn cria valores randômicos que seguem uma distribuição normal.
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        
        # Cria a matriz de valores randômicos para inicializar os pesos. 
        # A função randn cria valores randômicos que seguem uma distribuição normal.
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

        
    # Método feed forward
    # Este método faz a passada para frente até a previsão com a função sigmoid.
    def feedforward(self, a):
        
        # Utiliza os valores de peso e bias aprendidos no treinamento para fazer previsões com x
        for b, w in zip(self.biases, self.weights):
                        
            # Produto escalar e soma
            a = sigmoid(np.dot(w, a) + b)
        return a
        
    # Método SGD
    # Usado para treinar o modelo com algoritmo stochastic gradient descent
    def treina_modelo(self, training_data, epochs, mini_batch_size, taxa_aprendizado, test_data = None):

        # Recebe a matriz com os dados de entrada e converte em uma lista
        training_data = list(training_data)
        
        # Comprimento da matriz
        n = len(training_data)

        # Verifica se tem dados de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        # Loop pelo número de épocas (passadas de treinamento)
        for j in range(epochs):
            
            # Embaralha os dados de treino
            random.shuffle(training_data)
            
            # Extrai um mini-batch (lote) dos dados
            mini_batches = [training_data[k: k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            # Atualiza os pesos com o método atualiza_pesos
            for mini_batch in mini_batches:
                self.atualiza_pesos(mini_batch, taxa_aprendizado)
            
            # Imprime o andamento
            if test_data:
                print(f"Epoch {j} : {self.evaluate(test_data)} / {n_test}")
            else:
                print(f"Epoch {j} finalizada")
                
    # Método de atualização dos pesos
    # Atualiza pesos e bias conforme aprendizado do algoritmo
    def atualiza_pesos(self, mini_batch, taxa_aprendizado):
        
        # Cria as matrizes para os valores parciais de bias e pesos
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Para cada par de dados no lote, executa o aprendizado com o algoritmo backpropagation
        for x, y in mini_batch:
            
            # Calcula as derivadas parciais de pesos e bias
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            
            # Atualiza as matrizes parciais de pesos e bias com os novos valores
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        # Atualiza as matrizes principais de pesos e bias
        self.weights = [w - (taxa_aprendizado / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (taxa_aprendizado / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
    
    # Método do algoritmo backpropagation
    # Retorna tuplas representando o gradiente para a função de custo.
    def backprop(self, x, y):
        
        # Matrizes parciais
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as ativações, camada por camada
        activations = [x]

        # Lista para armazenar todos os vetores z, camada por camada
        zs = []

        # Loop pelas matrizes de pesos e bias
        for b, w in zip(self.biases, self.weights):
            
            # Produto escalar
            z = np.dot(w, activation) + b
                        
            # Append
            zs.append(z)
            
            # Ativação
            activation = sigmoid(z)
            
            # Append
            activations.append(activation)
        
        # Backward pass
        delta = self.derivada_custo(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Aqui, l = 1 significa a última camada de neurônios, l = 2 é a segunda e assim por diante.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        
        return (nabla_b, nabla_w)

    
    # Método de avaliação
    def evaluate(self, test_data):
        
        # Resultados de avaliação do modelo
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        
        return sum(int(x = y) for (x, y) in test_results)

    
    # Derivada do custo
    def derivada_custo(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations - y)
    
    # Função de Ativação Sigmóide
    def sigmoid(z):
        return 1.0/(1.0 + np.exp(-z))    

In [11]:
sizes = [2, 3, 1]

In [12]:
modelo_v1 = NeuralNetwork(sizes)

## Fim